In [1]:
import random
import pandas as pd
import numpy as np
# Read the CSV file into a pandas DataFrame
data = pd.read_csv("C:\\Users\\Mugun\\Desktop\\log.csv")

In [2]:
data = data.iloc[:-3]
print(data)

    x1   x2    y
0  3.0  2.0  0.0
1  3.0  3.0  0.0
2  5.0  1.0  1.0
3  1.0  3.0  0.0
4  2.0  1.0  0.0
5  6.0  4.0  0.0
6  3.0  1.0  1.0
7  1.0  5.0  0.0
8  5.0  1.0  1.0
9  1.0  1.0  0.0


In [3]:
def sig(x1, x2, w1, w2):
    z = w1 * x1 + w2 * x2
    return 1 / (1 + np.exp(-z))

def cost(x1,x2,w1,w2,y):
    si=sig(x1,x2,w1,w2)
    return -(y*np.log10(si+1e-30)+(1-y)*np.log10(1-si+1e-30))     #+1e-8

def aver(w1,w2):
    t=[cost(data.iloc[i].x1,data.iloc[i].x2,w1,w2,data.iloc[i].y) for i in range(10)]
    return sum(t)/len(t)
#aver(2.9979123689672518 , -7.001783182021187 )

# stocastic gradient descent

In [4]:
def sig(x1, x2, w1, w2):
    z = w1 * x1 + w2 * x2
    return 1 / (1 + np.exp(-z))

In [5]:
def ch(x1,x2,w1,w2,y,lr):
    dw1=(sig(x1, x2, w1, w2)-y)*x1
    dw2=(sig(x1, x2, w1, w2)-y)*x2
    w1-=lr*dw1
    w2-=lr*dw2
    return w1,w2

In [6]:
def gd(w1,w2,lr):
    l=[i for i in range(10)]
    random.shuffle(l)
    for i in l:
        w1,w2=ch(data.iloc[i].x1,data.iloc[i].x2,w1,w2,data.iloc[i].y,lr)
    return w1,w2

In [7]:
def percentage_difference(value1, value2):
    return ((value1 - value2)/((value1+value2)/2))*100

In [8]:
def check(w1,w2):
    print("y\ty^")
    for i in range(10):
        print(data.iloc[i].y,end="\t")
        if(sig(data.iloc[i].x1,data.iloc[i].x2,w1,w2)>0.8):
            print(1)
        else:
            print(0)
#check(2.7914059725033833,-6.754143440593903)

In [12]:
lr=0.1
w1=10
w2=10
av=aver(w1,w2)
bav=av
count=0
#print("1. w1=",w1," w2=",w2," avg=",av)
for j in range(1000):
    w1,w2=gd(w1,w2,lr)
    av=aver(w1,w2)
    #print(j+2,". w1=",w1," w2=",w2," avg=",av)
    if(percentage_difference(bav,av)<0.2):
        count+=1
    else:
        count=0
    if(count==10):
        break
    bav=av
print("Epoch : ",j+2,"\nw1 = ",w1,"\nw2 = ",w2,"\nAverage = ",av)
check(w1,w2)

Epoch :  308 
w1 =  2.8630342262523554 
w2 =  -6.940010162625523 
Average =  0.019934870020805726
y	y^
0.0	0
0.0	0
1.0	1
0.0	0
0.0	0
0.0	0
1.0	1
0.0	0
1.0	1
0.0	0


# Batch gradient descent

In [13]:
def sig(x1, x2, w1, w2):
    z = w1 * x1 + w2 * x2
    p=1 / (1 + np.exp(-z))
    return p

def bgd(w1,w2,lr):
    dw1=[(sig(data.iloc[i].x1,data.iloc[i].x2, w1, w2)-data.iloc[i].y)*data.iloc[i].x1 for i in range(10)]
    dw2=[(sig(data.iloc[i].x1,data.iloc[i].x2, w1, w2)-data.iloc[i].y)*data.iloc[i].x2 for i in range(10)]
    w1-=lr*(sum(dw1)/len(dw1))
    w2-=lr*(sum(dw2)/len(dw2))
    return w1,w2

In [14]:
def percentage_difference(value1, value2):
    return ((value1 - value2)/((value1+value2)/2))*100

In [15]:
def check(w1,w2):
    print("y\ty^")
    for i in range(10):
        print(data.iloc[i].y,end="\t")
        if(sig(data.iloc[i].x1,data.iloc[i].x2,w1,w2)>0.8):
            print(1)
        else:
            print(0)
#check(3.423758849003598, -8.372762480958373)

In [16]:
lr=0.2
w1=10
w2=10
av=aver(w1,w2)
bav=av
count=0
#print("1. w1=",w1," w2=",w2," avg=",av)
for j in range(3500):
    w1,w2=bgd(w1,w2,lr)
    av=aver(w1,w2)
    #print(j+2,". w1=",w1," w2=",w2," avg=",av)
    if(percentage_difference(bav,av)<0.03):
        count+=1
    else:
        count=0
    if(count==10):
        break
    bav=av
print("Epoch : ",j+2,"\nw1 = ",w1,"\nw2 = ",w2,"\nAverage = ",av)
check(w1,w2)

Epoch :  1827 
w1 =  2.97885152996468 
w2 =  -7.266453896158529 
Average =  0.018672160867094077
y	y^
0.0	0
0.0	0
1.0	1
0.0	0
0.0	0
0.0	0
1.0	1
0.0	0
1.0	1
0.0	0


# Mini Batch Gradient descent

In [17]:
def mbgd(w1,w2,lr):
    bs=5
    l=[i for i in range(10)]
    random.shuffle(l)
    for i in range(0,len(l),bs):
        l1=l[i:i+bs]
        dw1=[(sig(data.iloc[i].x1,data.iloc[i].x2, w1, w2)-data.iloc[i].y)*data.iloc[i].x1 for i in l1]
        dw2=[(sig(data.iloc[i].x1,data.iloc[i].x2, w1, w2)-data.iloc[i].y)*data.iloc[i].x2 for i in l1]
        w1-=lr*(sum(dw1)/len(dw1))
        w2-=lr*(sum(dw2)/len(dw2))
        #w1,w2=ch(data.iloc[i].x1,data.iloc[i].x2,w1,w2,data.iloc[i].y,lr)
    return w1,w2

In [18]:
def percentage_difference(value1, value2):
    return ((value1 - value2)/((value1+value2)/2))*100

In [19]:
def check(w1,w2):
    print("y\ty^")
    for i in range(10):
        print(data.iloc[i].y,end="\t")
        if(sig(data.iloc[i].x1,data.iloc[i].x2,w1,w2)>0.8):
            print(1)
        else:
            print(0)

In [20]:
lr=0.1
w1=10
w2=10
av=aver(w1,w2)
bav=av
count=0
#print("1. w1=",w1," w2=",w2," avg=",av)
for j in range(3000):
    w1,w2=mbgd(w1,w2,lr)
    av=aver(w1,w2)
    #print(j+2,". w1=",w1," w2=",w2," avg=",av)
    if(percentage_difference(bav,av)<0.03):
        count+=1
    else:
        count=0
    if(count==10):
        break
    bav=av
print("Epoch : ",j+2,"\nw1 = ",w1,"\nw2 = ",w2,"\nAverage = ",av)
check(w1,w2)

Epoch :  1843 
w1 =  2.9939120967953223 
w2 =  -7.3045764383925444 
Average =  0.018530143863240013
y	y^
0.0	0
0.0	0
1.0	1
0.0	0
0.0	0
0.0	0
1.0	1
0.0	0
1.0	1
0.0	0
